In [1]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())


In [3]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [4]:
class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [7]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


In [8]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
for epoch in range(1, 10):
    train(epoch)
    test()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303620
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.287684
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.300245
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.296173
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.300041
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.294008
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.292836
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.280002
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.304977
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.288180
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.279792
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.258584
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.234388
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.224407
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.193866
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.153205
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.113793
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.027592
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.844917
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.460556
Train Epoch: 1 [12800/60000 (

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if __name__ == '__main__':



Test set: Average loss: 0.1847, Accuracy: 9465/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.407396
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.194831
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.116162
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.137398
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.289264
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.220973
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.096004
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.177293
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.289889
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.154643
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.059723
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.097633
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.089331
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.168689
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.163649
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.072027
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.190322
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.179340
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.122382
Train Epoch: 2 [12

Train Epoch: 3 [43520/60000 (72%)]	Loss: 0.088377
Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.070930
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.135814
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.133670
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.033505
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.094372
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.030056
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.076652
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.088156
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.031673
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.092614
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.171948
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.089555
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.071673
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.017394
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.495990
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.043459
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.095501
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.110112
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.115137


Train Epoch: 5 [26880/60000 (45%)]	Loss: 0.049796
Train Epoch: 5 [27520/60000 (46%)]	Loss: 0.039935
Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.004124
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.024338
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.034066
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.026066
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.058810
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.070934
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.026055
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.138437
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.028094
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.016202
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.047676
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.047777
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.093601
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.016391
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.066199
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.106929
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.050910
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.104107


Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.165182
Train Epoch: 7 [10880/60000 (18%)]	Loss: 0.057313
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.077700
Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.024995
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.095754
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.043067
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.089149
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.062413
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.018450
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.009002
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.022322
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.089927
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.131869
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.142825
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.050248
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.011084
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.019881
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.026394
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.102972
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.074369


Train Epoch: 8 [54400/60000 (91%)]	Loss: 0.039234
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.040446
Train Epoch: 8 [55680/60000 (93%)]	Loss: 0.065823
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.018029
Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.054084
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.063861
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.006745
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.018331
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.045044

Test set: Average loss: 0.0545, Accuracy: 9824/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.016607
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.004304
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.112117
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.022072
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.038867
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.097137
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.078373
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.102470
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.041822
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.009976
Train Epoch: